In [1]:
from ipynb.fs.full.utils import *

import numpy as np
import h5py
import pandas as pd
import sys
import illustris_python.illustris_python as il
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from itertools import repeat
import tqdm

In [2]:
def get_insitu_stellar_particle_ids(basePath, subhalo_id, snapshot, previous_snapshot_time):
    formed_insitu, particle_ids, masses = set(), set(), []
    fields = ['ParticleIDs', 'GFM_StellarFormationTime', 'Masses']   
    stars = il.snapshot.loadSubhalo(basePath, snapshot, subhalo_id,
                                    'stars', fields=fields)
    
    header = il.groupcat.loadHeader(basePath, snapshot)
    snapshot_time = header['Time']
    if stars['count'] > 0:
        particle_ids = stars['ParticleIDs']
        formation_time = stars['GFM_StellarFormationTime']
        masses = stars['Masses']
        formed_insitu = set(particle_ids[np.where((previous_snapshot_time < formation_time[:]) & 
                                                  (formation_time[:] <= snapshot_time))])
        particle_ids = particle_ids[formation_time[:] > 0]
        masses = masses[formation_time[:] > 0]
    return formed_insitu, snapshot_time, particle_ids, masses



In [3]:
def get_exsitu_fraction(subhalo_id, snapshot, sim_dir):
    basePath = '/home/jovyan/Data/Sims/IllustrisTNG/{}/'.format(sim_dir)
    fields = ['SubhaloMass','SubfindID','SnapNum']
    tree = il.sublink.loadTree(basePath, snapshot, subhalo_id, fields=fields, onlyMPB=True) 

    insitu_overall = set()
    previous = 0
    for i in reversed(range(len(tree['SnapNum'][:]))):
        insitu, previous, all_ids, masses = get_insitu_stellar_particle_ids(basePath, 
                                                                            tree['SubfindID'][i],
                                                                            tree['SnapNum'][i],
                                                                            previous)
        insitu_overall.update(insitu)


    insitu_overall = insitu_overall.intersection(all_ids)
    insitu_mass = sum(m for i, m in enumerate(masses) if all_ids[i] in insitu_overall)
    total_mass = sum(masses)
    
    exsitu_f = -1
    if total_mass > 0:
        exsitu_f = (total_mass - insitu_mass)/total_mass
    
    return exsitu_f




In [4]:
def calc_exsitu_sim(sim_dir):
    basePath = '/home/jovyan/Data/Sims/IllustrisTNG/{}/'.format(sim_dir)
    exsitu_dir = 'Exsitu/IllustrisTNG/{}'.format(sim_dir)
    if not os.path.exists(exsitu_dir):
        os.makedirs(exsitu_dir)

    for snapshot in range(29, 34):
        (scaling_factor, hubble_param,
            boxsize, omega0, omegaLambda) = get_snapshot_values(basePath, snapshot)
        tng = get_catalog_data(basePath, snapshot, scaling_factor, hubble_param)
        galaxy_ids = list(tng.GalaxyID)

        pool = Pool(processes=4)
        results = list(tqdm.tqdm(pool.starmap(get_exsitu_fraction, 
                                              zip(galaxy_ids, repeat(snapshot), repeat(sim_dir))), 
                                 total=len(galaxy_ids)))


        pool.close()
        pool.join()

        exsitu_dict = {'GalaxyID': galaxy_ids, 'ExsituF': results}
        df = pd.DataFrame(exsitu_dict)

        save_path = os.path.join(exsitu_dir, 'exsitu_%03d.hdf5' % snapshot)
        df.to_hdf(save_path, '/data')
